Import libraries

In [85]:
import numpy as np
import pandas as pd

Load and read file

In [86]:
data = pd.read_csv('https://raw.githubusercontent.com/TracyRenee61/Bananas/main/bananas-11jan21.csv', encoding= 'unicode_escape')
data

,Origin,Date,Price,Units
0,belize,2021-01-10,0.86,£/kg
1,costa_rica,2021-01-10,0.87,£/kg
2,ivory_coast,2021-01-10,0.73,£/kg
3,acp_bananas,2021-01-10,0.81,£/kg
4,dollar_bananas,2021-01-10,0.87,£/kg
...,...,...,...,...
11791,guatemala,1995-01-20,0.64,£/kg
11792,guadeloupe,1995-01-20,0.61,£/kg
11793,surinam,1995-01-13,0.61,£/kg
11794,windward_isles,1995-01-13,0.60,£/kg


Sort dataframe by date

In [87]:
data = data.sort_values(by='Date')
data

,Origin,Date,Price,Units
11795,guadeloupe,1995-01-13,0.60,£/kg
11794,windward_isles,1995-01-13,0.60,£/kg
11793,surinam,1995-01-13,0.61,£/kg
11789,surinam,1995-01-20,0.63,£/kg
11792,guadeloupe,1995-01-20,0.61,£/kg
...,...,...,...,...
4,dollar_bananas,2021-01-10,0.87,£/kg
3,acp_bananas,2021-01-10,0.81,£/kg
2,ivory_coast,2021-01-10,0.73,£/kg
1,costa_rica,2021-01-10,0.87,£/kg


Count Origin

In [88]:
origin = data['Origin'].to_list()

# using naive method to remove duplicated from list  
list_origin = [] 
for i in origin: 
    if i not in list_origin: 
        list_origin.append(i) 

sorted(list_origin)

['acp_bananas',
 'all_bananas',
 'belize',
 'brazil',
 'cameroon',
 'colombia',
 'costa_rica',
 'dollar_bananas',
 'dominican_republic',
 'ecuador',
 'eu_bananas',
 'ghana',
 'guadeloupe',
 'guatemala',
 'honduras',
 'ivory_coast',
 'jamaica',
 'malaysia',
 'martinique',
 'mexico',
 'nicaragua',
 'panama',
 'somalia',
 'st_vincent',
 'surinam',
 'venezuela',
 'windward_isles']

In [89]:
print(len(list_origin))

27


Dates

In [90]:
from datetime import date 
from datetime import timedelta

In [91]:
max_date = data.Date.max()
max_date = pd.Timestamp(max_date)
max_date

Timestamp('2021-01-10 00:00:00')

Rename columns

In [92]:
data = data.drop('Units', axis=1)

df = data.rename(columns={'Date': 'ds', 'Origin': 'Group', 'Price':'y'})
df

,Group,ds,y
11795,guadeloupe,1995-01-13,0.60
11794,windward_isles,1995-01-13,0.60
11793,surinam,1995-01-13,0.61
11789,surinam,1995-01-20,0.63
11792,guadeloupe,1995-01-20,0.61
...,...,...,...
4,dollar_bananas,2021-01-10,0.87
3,acp_bananas,2021-01-10,0.81
2,ivory_coast,2021-01-10,0.73
1,costa_rica,2021-01-10,0.87


FB Prophet Predictions on each Group

In [93]:
from fbprophet import Prophet

final = pd.DataFrame()

grouped = df.groupby('Group')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet()
    m.fit(group)
    future = m.make_future_dataframe(periods=52, freq='w')
    forecast = m.predict(future)
    #I added a column with group
    forecast['group'] = g
    #I used concat instead of merge
    final = pd.concat([final, forecast], ignore_index=True)
final

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasona

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,group,weekly,weekly_lower,weekly_upper
0,1998-01-09,0.542255,0.371777,0.636506,0.542255,0.542255,-0.037939,-0.037939,-0.037939,-0.037939,-0.037939,-0.037939,0.0,0.0,0.0,0.504317,acp_bananas,NaN,NaN,NaN
1,1998-01-16,0.542182,0.438651,0.714604,0.542182,0.542182,0.032931,0.032931,0.032931,0.032931,0.032931,0.032931,0.0,0.0,0.0,0.575113,acp_bananas,NaN,NaN,NaN
2,1998-01-23,0.542108,0.497842,0.766023,0.542108,0.542108,0.085270,0.085270,0.085270,0.085270,0.085270,0.085270,0.0,0.0,0.0,0.627379,acp_bananas,NaN,NaN,NaN
3,1998-01-30,0.542035,0.510554,0.782063,0.542035,0.542035,0.100562,0.100562,0.100562,0.100562,0.100562,0.100562,0.0,0.0,0.0,0.642597,acp_bananas,NaN,NaN,NaN
4,1998-02-06,0.541962,0.486320,0.761807,0.541962,0.541962,0.083364,0.083364,0.083364,0.083364,0.083364,0.083364,0.0,0.0,0.0,0.625325,acp_bananas,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13195,2021-04-25,0.456233,0.365756,0.669636,0.454530,0.457876,0.057696,0.057696,0.057696,0.057696,0.057696,0.057696,0.0,0.0,0.0,0.513929,windward_isles,NaN,NaN,NaN
13196,2021-05-02,0.456046,0.358945,0.681662,0.454261,0.457731,0.057812,0.057812,0.057812,0.057812,0.057812,0.057812,0.0,0.0,0.0,0.513859,windward_isles,NaN,NaN,NaN
13197,2021-05-09,0.455859,0.379807,0.684554,0.453989,0.457596,0.070049,0.070049,0.070049,0.070049,0.070049,0.070049,0.0,0.0,0.0,0.525909,windward_isles,NaN,NaN,NaN
13198,2021-05-16,0.455673,0.370805,0.688791,0.453712,0.457522,0.073336,0.073336,0.073336,0.073336,0.073336,0.073336,0.0,0.0,0.0,0.529008,windward_isles,NaN,NaN,NaN


In [94]:
prediction = pd.DataFrame({'date': final.ds, 'prediction': final.yhat, 'group':final.group })
prediction

,date,prediction,group
0,1998-01-09,0.504317,acp_bananas
1,1998-01-16,0.575113,acp_bananas
2,1998-01-23,0.627379,acp_bananas
3,1998-01-30,0.642597,acp_bananas
4,1998-02-06,0.625325,acp_bananas
...,...,...,...
13195,2021-04-25,0.513929,windward_isles
13196,2021-05-02,0.513859,windward_isles
13197,2021-05-09,0.525909,windward_isles
13198,2021-05-16,0.529008,windward_isles


In [95]:
future_prediction = prediction[(prediction['date'] > max_date)]
future_prediction

,date,prediction,group
1105,2021-01-17,0.681049,acp_bananas
1106,2021-01-24,0.727382,acp_bananas
1107,2021-01-31,0.735787,acp_bananas
1108,2021-02-07,0.714616,acp_bananas
1109,2021-02-14,0.685668,acp_bananas
...,...,...,...
13195,2021-04-25,0.513929,windward_isles
13196,2021-05-02,0.513859,windward_isles
13197,2021-05-09,0.525909,windward_isles
13198,2021-05-16,0.529008,windward_isles


In [96]:
origin = future_prediction['group'].to_list()

# using naive method to remove duplicated from list  
list_origin = [] 
for i in origin: 
    if i not in list_origin: 
        list_origin.append(i) 

sorted(list_origin)

['acp_bananas',
 'all_bananas',
 'belize',
 'colombia',
 'costa_rica',
 'dollar_bananas',
 'dominican_republic',
 'ecuador',
 'ghana',
 'guatemala',
 'honduras',
 'ivory_coast',
 'mexico',
 'nicaragua',
 'panama',
 'windward_isles']

In [97]:
print(len(list_origin))

16


Select group


In [98]:
origin = 'dollar_bananas'

single_prediction = future_prediction[(future_prediction['group'] == origin)]
single_prediction


,date,prediction,group
6698,2021-01-17,0.917971,dollar_bananas
6699,2021-01-24,0.958536,dollar_bananas
6700,2021-01-31,0.970835,dollar_bananas
6701,2021-02-07,0.959788,dollar_bananas
6702,2021-02-14,0.941870,dollar_bananas
6703,2021-02-21,0.932674,dollar_bananas
6704,2021-02-28,0.937733,dollar_bananas
6705,2021-03-07,0.952613,dollar_bananas
6706,2021-03-14,0.969127,dollar_bananas
6707,2021-03-21,0.980476,dollar_bananas
